# Image classification task
- Image classification on OpenML Task (362128), tiniest ImageNet dataset.

In [ ]:
import torch.nn
import torch.optim

import openml_pytorch.config
import openml
import logging
import warnings

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore')

############################################################################
# Enable logging in order to observe the progress while running the example.
openml.config.logger.setLevel(logging.DEBUG)
openml_pytorch.config.logger.setLevel(logging.DEBUG)
############################################################################

############################################################################
from openml_pytorch.trainer import OpenMLTrainerModule
from openml_pytorch.trainer import OpenMLDataModule
from torchvision.transforms import Compose, Resize, ToPILImage, ToTensor, Lambda
import torchvision

from openml_pytorch.trainer import convert_to_rgb

## Define the Model

In [ ]:
model = torchvision.models.efficientnet_b0(num_classes=200)

## Configure the Data Module
- Make sure the data is present in the `file_dir` directory, and the `filename_col` is correctly set along with this column correctly pointing to where your data is stored. 

In [ ]:
transform = Compose(
    [
        ToPILImage(),  # Convert tensor to PIL Image to ensure PIL Image operations can be applied.
        Lambda(
            convert_to_rgb
        ),  # Convert PIL Image to RGB if it's not already.
        Resize(
            (64, 64)
        ),  # Resize the image.
        ToTensor(),  # Convert the PIL Image back to a tensor.
    ]
)
data_module = OpenMLDataModule(
    type_of_data="image",
    file_dir="datasets",
    filename_col="image_path",
    target_mode="categorical",
    target_column="label",
    batch_size = 64,
    transform=transform
)

## Configure the Trainer Module

In [ ]:
trainer = OpenMLTrainerModule(
    data_module=data_module,
    verbose = True,
    epoch_count = 1,
    callbacks=[],
)
openml_pytorch.config.trainer = trainer

## Download the task

In [ ]:
# Download the OpenML task for tiniest imagenet
task = openml.tasks.get_task(362128)

## Run the model on the task

In [ ]:
run = openml.runs.run_model_on_task(model, task, avoid_duplicate_runs=False)

In [ ]:
run.publish()

## View loss

In [ ]:
trainer.runner.cbs[1].plot_loss()

## View learning rate

In [ ]:
trainer.runner.cbs[1].plot_lr()

## View the classes in the model

In [ ]:
trainer.learn.model_classes

## Publish the run to OpenML

In [ ]:
run.publish()